In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [4]:
signatures = pd.read_csv('../denoising/sliding_window_y.csv', index_col=0)
signals = pd.read_csv('../denoising/sliding_window_X.csv', index_col=0)

In [5]:
signals = np.asarray(signals)
signatures = np.asarray(signatures)
sample_length = 32

In [6]:
y = np.zeros((len(signatures),2))

for i in range(len(signatures)):
    pos = np.where(signatures[i] > 0)[0]
    if len(pos) > 0:
        y[i][0] += np.max([0, pos[0] - 1])
        y[i][1] += np.min([sample_length, pos[-1] + 1])

In [7]:
X = signals[:,:,np.newaxis]

mean = X.mean(axis=1).reshape((X.shape[0],1,1))
#mean = 0.0
X = X - mean
sd = X.std(axis=1).mean()
#rand_sd = rand_sd.sample(frac=1).reset_index(drop=True)
X /= sd
print("Mean: ", X.mean())
print("Std: ", X.std())


Mean:  -7.99317374239e-17
Std:  1.21030715591


In [8]:
pd.DataFrame([[sd]],columns=['sd']).to_csv('normalization_params_cl.csv')

In [9]:
y_n = np.zeros(len(y))

for i in range(len(y)):
    if y[i][0] + y[i][1] > 0:
        y_n[i] += 1
y_n

array([ 0.,  1.,  1., ...,  0.,  0.,  0.])

In [10]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# Forcing tf to use the CPU
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
set_session(tf.Session(config=config))

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
# set_session(tf.Session(config=config))

Using TensorFlow backend.


In [27]:
from keras.models import load_model

model = load_model('rectangles.h5')

In [46]:
from keras.layers import Dense
model2 = model
model2.add(Dense(units=1,activation='sigmoid'))

AttributeError: 'Sequential' object has no attribute 'copy'

In [28]:
model2.compile(optimizer='adagrad',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model2.fit(X, y_n, batch_size=64,verbose=1,epochs=1, validation_split=0.2)

Train on 391219 samples, validate on 97805 samples
Epoch 1/1


 12928/391219 [..............................] - ETA: 1:55:03 - loss: 0.7431 - acc: 0.48 - ETA: 1:18:54 - loss: 0.7326 - acc: 0.45 - ETA: 1:06:13 - loss: 0.7009 - acc: 0.51 - ETA: 59:53 - loss: 0.6776 - acc: 0.5625 - ETA: 56:14 - loss: 0.6627 - acc: 0.59 - ETA: 53:32 - loss: 0.6667 - acc: 0.58 - ETA: 52:07 - loss: 0.6588 - acc: 0.59 - ETA: 50:27 - loss: 0.6558 - acc: 0.60 - ETA: 49:35 - loss: 0.6505 - acc: 0.61 - ETA: 48:51 - loss: 0.6382 - acc: 0.63 - ETA: 48:05 - loss: 0.6304 - acc: 0.64 - ETA: 47:35 - loss: 0.6236 - acc: 0.65 - ETA: 47:05 - loss: 0.6273 - acc: 0.65 - ETA: 46:42 - loss: 0.6249 - acc: 0.65 - ETA: 46:18 - loss: 0.6220 - acc: 0.66 - ETA: 46:03 - loss: 0.6178 - acc: 0.66 - ETA: 45:48 - loss: 0.6188 - acc: 0.66 - ETA: 45:31 - loss: 0.6148 - acc: 0.67 - ETA: 45:17 - loss: 0.6158 - acc: 0.67 - ETA: 45:01 - loss: 0.6120 - acc: 0.67 - ETA: 44:53 - loss: 0.6101 - acc: 0.68 - ETA: 44:41 - loss: 0.6086 - acc: 0.68 - ETA: 44:30 - loss: 0.6060 - acc: 0.68 - ETA: 44:28 - loss: 0.60

 39040/391219 [=>............................] - ETA: 41:19 - loss: 0.4929 - acc: 0.82 - ETA: 41:19 - loss: 0.4929 - acc: 0.82 - ETA: 41:18 - loss: 0.4929 - acc: 0.82 - ETA: 41:17 - loss: 0.4927 - acc: 0.82 - ETA: 41:16 - loss: 0.4928 - acc: 0.82 - ETA: 41:15 - loss: 0.4925 - acc: 0.82 - ETA: 41:14 - loss: 0.4925 - acc: 0.82 - ETA: 41:13 - loss: 0.4925 - acc: 0.82 - ETA: 41:12 - loss: 0.4923 - acc: 0.82 - ETA: 41:12 - loss: 0.4922 - acc: 0.82 - ETA: 41:11 - loss: 0.4921 - acc: 0.82 - ETA: 41:10 - loss: 0.4919 - acc: 0.82 - ETA: 41:10 - loss: 0.4918 - acc: 0.82 - ETA: 41:09 - loss: 0.4915 - acc: 0.82 - ETA: 41:08 - loss: 0.4917 - acc: 0.82 - ETA: 41:08 - loss: 0.4915 - acc: 0.82 - ETA: 41:08 - loss: 0.4916 - acc: 0.82 - ETA: 41:07 - loss: 0.4916 - acc: 0.82 - ETA: 41:06 - loss: 0.4918 - acc: 0.82 - ETA: 41:07 - loss: 0.4918 - acc: 0.82 - ETA: 41:08 - loss: 0.4916 - acc: 0.82 - ETA: 41:08 - loss: 0.4915 - acc: 0.82 - ETA: 41:10 - loss: 0.4914 - acc: 0.82 - ETA: 41:11 - loss: 0.4912 - acc

 52096/391219 [==>...........................] - ETA: 39:24 - loss: 0.4741 - acc: 0.84 - ETA: 39:24 - loss: 0.4739 - acc: 0.84 - ETA: 39:23 - loss: 0.4738 - acc: 0.84 - ETA: 39:22 - loss: 0.4738 - acc: 0.84 - ETA: 39:22 - loss: 0.4738 - acc: 0.84 - ETA: 39:21 - loss: 0.4737 - acc: 0.84 - ETA: 39:20 - loss: 0.4737 - acc: 0.84 - ETA: 39:20 - loss: 0.4738 - acc: 0.84 - ETA: 39:19 - loss: 0.4737 - acc: 0.84 - ETA: 39:19 - loss: 0.4735 - acc: 0.84 - ETA: 39:18 - loss: 0.4734 - acc: 0.84 - ETA: 39:17 - loss: 0.4734 - acc: 0.84 - ETA: 39:16 - loss: 0.4733 - acc: 0.84 - ETA: 39:16 - loss: 0.4732 - acc: 0.84 - ETA: 39:15 - loss: 0.4731 - acc: 0.84 - ETA: 39:15 - loss: 0.4730 - acc: 0.84 - ETA: 39:14 - loss: 0.4728 - acc: 0.84 - ETA: 39:13 - loss: 0.4727 - acc: 0.84 - ETA: 39:13 - loss: 0.4727 - acc: 0.84 - ETA: 39:12 - loss: 0.4725 - acc: 0.84 - ETA: 39:11 - loss: 0.4724 - acc: 0.84 - ETA: 39:11 - loss: 0.4723 - acc: 0.84 - ETA: 39:10 - loss: 0.4723 - acc: 0.84 - ETA: 39:09 - loss: 0.4722 - acc

 65152/391219 [===>..........................] - ETA: 37:37 - loss: 0.4600 - acc: 0.85 - ETA: 37:37 - loss: 0.4599 - acc: 0.85 - ETA: 37:36 - loss: 0.4599 - acc: 0.85 - ETA: 37:36 - loss: 0.4597 - acc: 0.85 - ETA: 37:36 - loss: 0.4598 - acc: 0.85 - ETA: 37:35 - loss: 0.4597 - acc: 0.85 - ETA: 37:34 - loss: 0.4598 - acc: 0.85 - ETA: 37:34 - loss: 0.4598 - acc: 0.85 - ETA: 37:34 - loss: 0.4597 - acc: 0.85 - ETA: 37:33 - loss: 0.4598 - acc: 0.85 - ETA: 37:33 - loss: 0.4598 - acc: 0.85 - ETA: 37:32 - loss: 0.4595 - acc: 0.85 - ETA: 37:32 - loss: 0.4594 - acc: 0.85 - ETA: 37:31 - loss: 0.4594 - acc: 0.85 - ETA: 37:31 - loss: 0.4593 - acc: 0.85 - ETA: 37:30 - loss: 0.4593 - acc: 0.85 - ETA: 37:30 - loss: 0.4592 - acc: 0.85 - ETA: 37:30 - loss: 0.4592 - acc: 0.85 - ETA: 37:29 - loss: 0.4590 - acc: 0.85 - ETA: 37:29 - loss: 0.4589 - acc: 0.85 - ETA: 37:28 - loss: 0.4589 - acc: 0.85 - ETA: 37:28 - loss: 0.4588 - acc: 0.85 - ETA: 37:27 - loss: 0.4587 - acc: 0.85 - ETA: 37:26 - loss: 0.4586 - acc

...

339328/391219 [=========================>....] - ETA: 7:02 - loss: 0.3661 - acc: 0.887 - ETA: 7:01 - loss: 0.3661 - acc: 0.887 - ETA: 7:01 - loss: 0.3661 - acc: 0.887 - ETA: 7:00 - loss: 0.3661 - acc: 0.887 - ETA: 7:00 - loss: 0.3661 - acc: 0.887 - ETA: 7:00 - loss: 0.3661 - acc: 0.887 - ETA: 6:59 - loss: 0.3661 - acc: 0.887 - ETA: 6:59 - loss: 0.3661 - acc: 0.887 - ETA: 6:58 - loss: 0.3661 - acc: 0.887 - ETA: 6:58 - loss: 0.3660 - acc: 0.887 - ETA: 6:58 - loss: 0.3660 - acc: 0.887 - ETA: 6:57 - loss: 0.3660 - acc: 0.887 - ETA: 6:57 - loss: 0.3660 - acc: 0.887 - ETA: 6:56 - loss: 0.3660 - acc: 0.887 - ETA: 6:56 - loss: 0.3660 - acc: 0.887 - ETA: 6:56 - loss: 0.3660 - acc: 0.887 - ETA: 6:55 - loss: 0.3660 - acc: 0.887 - ETA: 6:55 - loss: 0.3660 - acc: 0.887 - ETA: 6:54 - loss: 0.3660 - acc: 0.887 - ETA: 6:54 - loss: 0.3659 - acc: 0.887 - ETA: 6:53 - loss: 0.3659 - acc: 0.887 - ETA: 6:53 - loss: 0.3659 - acc: 0.887 - ETA: 6:53 - loss: 0.3659 - acc: 0.887 - ETA: 6:52 - loss: 0.3659 - acc:

352384/391219 [==========================>...] - ETA: 5:37 - loss: 0.3639 - acc: 0.888 - ETA: 5:36 - loss: 0.3639 - acc: 0.888 - ETA: 5:36 - loss: 0.3639 - acc: 0.888 - ETA: 5:35 - loss: 0.3639 - acc: 0.888 - ETA: 5:35 - loss: 0.3639 - acc: 0.888 - ETA: 5:34 - loss: 0.3639 - acc: 0.888 - ETA: 5:34 - loss: 0.3639 - acc: 0.888 - ETA: 5:34 - loss: 0.3638 - acc: 0.888 - ETA: 5:33 - loss: 0.3638 - acc: 0.888 - ETA: 5:33 - loss: 0.3638 - acc: 0.888 - ETA: 5:32 - loss: 0.3638 - acc: 0.888 - ETA: 5:32 - loss: 0.3638 - acc: 0.888 - ETA: 5:32 - loss: 0.3638 - acc: 0.888 - ETA: 5:31 - loss: 0.3638 - acc: 0.888 - ETA: 5:31 - loss: 0.3637 - acc: 0.888 - ETA: 5:30 - loss: 0.3637 - acc: 0.888 - ETA: 5:30 - loss: 0.3638 - acc: 0.888 - ETA: 5:29 - loss: 0.3637 - acc: 0.888 - ETA: 5:29 - loss: 0.3637 - acc: 0.888 - ETA: 5:29 - loss: 0.3637 - acc: 0.888 - ETA: 5:28 - loss: 0.3637 - acc: 0.888 - ETA: 5:28 - loss: 0.3637 - acc: 0.888 - ETA: 5:27 - loss: 0.3637 - acc: 0.888 - ETA: 5:27 - loss: 0.3637 - acc:

365440/391219 [===========================>..] - ETA: 4:11 - loss: 0.3618 - acc: 0.888 - ETA: 4:11 - loss: 0.3618 - acc: 0.888 - ETA: 4:11 - loss: 0.3618 - acc: 0.888 - ETA: 4:10 - loss: 0.3618 - acc: 0.888 - ETA: 4:10 - loss: 0.3618 - acc: 0.888 - ETA: 4:09 - loss: 0.3617 - acc: 0.888 - ETA: 4:09 - loss: 0.3617 - acc: 0.888 - ETA: 4:09 - loss: 0.3617 - acc: 0.888 - ETA: 4:08 - loss: 0.3617 - acc: 0.888 - ETA: 4:08 - loss: 0.3617 - acc: 0.888 - ETA: 4:07 - loss: 0.3617 - acc: 0.888 - ETA: 4:07 - loss: 0.3617 - acc: 0.888 - ETA: 4:06 - loss: 0.3617 - acc: 0.888 - ETA: 4:06 - loss: 0.3617 - acc: 0.888 - ETA: 4:06 - loss: 0.3617 - acc: 0.888 - ETA: 4:05 - loss: 0.3616 - acc: 0.888 - ETA: 4:05 - loss: 0.3616 - acc: 0.888 - ETA: 4:04 - loss: 0.3616 - acc: 0.888 - ETA: 4:04 - loss: 0.3616 - acc: 0.888 - ETA: 4:04 - loss: 0.3616 - acc: 0.888 - ETA: 4:03 - loss: 0.3616 - acc: 0.888 - ETA: 4:03 - loss: 0.3616 - acc: 0.888 - ETA: 4:02 - loss: 0.3616 - acc: 0.888 - ETA: 4:02 - loss: 0.3615 - acc:

378496/391219 [============================>.] - ETA: 2:46 - loss: 0.3598 - acc: 0.889 - ETA: 2:46 - loss: 0.3598 - acc: 0.889 - ETA: 2:46 - loss: 0.3598 - acc: 0.889 - ETA: 2:45 - loss: 0.3597 - acc: 0.889 - ETA: 2:45 - loss: 0.3597 - acc: 0.889 - ETA: 2:44 - loss: 0.3597 - acc: 0.889 - ETA: 2:44 - loss: 0.3597 - acc: 0.889 - ETA: 2:44 - loss: 0.3597 - acc: 0.889 - ETA: 2:43 - loss: 0.3597 - acc: 0.889 - ETA: 2:43 - loss: 0.3597 - acc: 0.889 - ETA: 2:42 - loss: 0.3597 - acc: 0.889 - ETA: 2:42 - loss: 0.3597 - acc: 0.889 - ETA: 2:41 - loss: 0.3597 - acc: 0.889 - ETA: 2:41 - loss: 0.3597 - acc: 0.889 - ETA: 2:41 - loss: 0.3597 - acc: 0.889 - ETA: 2:40 - loss: 0.3597 - acc: 0.889 - ETA: 2:40 - loss: 0.3597 - acc: 0.889 - ETA: 2:39 - loss: 0.3596 - acc: 0.889 - ETA: 2:39 - loss: 0.3596 - acc: 0.889 - ETA: 2:39 - loss: 0.3596 - acc: 0.889 - ETA: 2:38 - loss: 0.3596 - acc: 0.889 - ETA: 2:38 - loss: 0.3596 - acc: 0.889 - ETA: 2:37 - loss: 0.3596 - acc: 0.889 - ETA: 2:37 - loss: 0.3596 - acc:

391219/391219 [==============================] - ETA: 1:22 - loss: 0.3579 - acc: 0.890 - ETA: 1:21 - loss: 0.3579 - acc: 0.890 - ETA: 1:21 - loss: 0.3579 - acc: 0.890 - ETA: 1:20 - loss: 0.3579 - acc: 0.890 - ETA: 1:20 - loss: 0.3579 - acc: 0.890 - ETA: 1:20 - loss: 0.3579 - acc: 0.890 - ETA: 1:19 - loss: 0.3579 - acc: 0.890 - ETA: 1:19 - loss: 0.3578 - acc: 0.890 - ETA: 1:18 - loss: 0.3578 - acc: 0.890 - ETA: 1:18 - loss: 0.3578 - acc: 0.890 - ETA: 1:17 - loss: 0.3578 - acc: 0.890 - ETA: 1:17 - loss: 0.3578 - acc: 0.890 - ETA: 1:17 - loss: 0.3578 - acc: 0.890 - ETA: 1:16 - loss: 0.3577 - acc: 0.890 - ETA: 1:16 - loss: 0.3577 - acc: 0.890 - ETA: 1:15 - loss: 0.3577 - acc: 0.890 - ETA: 1:15 - loss: 0.3577 - acc: 0.890 - ETA: 1:15 - loss: 0.3577 - acc: 0.890 - ETA: 1:14 - loss: 0.3577 - acc: 0.890 - ETA: 1:14 - loss: 0.3577 - acc: 0.890 - ETA: 1:13 - loss: 0.3577 - acc: 0.890 - ETA: 1:13 - loss: 0.3576 - acc: 0.890 - ETA: 1:13 - loss: 0.3577 - acc: 0.890 - ETA: 1:12 - loss: 0.3576 - acc:

In [29]:
pred2 = model2.predict(X)

In [34]:
pred2_class = np.zeros(len(pred2))
pred2_class[np.where(pred2 >= 0.5)[0]] += 1
truth_class = np.zeros(len(pred2))
truth_class[np.where(np.sum(signatures,axis=1) > 0)] += 1

NameError: name 'pred2' is not defined

In [37]:
precision = len(np.where((pred2_class == 1) & (truth_class == 1))[0]) / len(np.where((pred2_class == 1))[0])
recall = len(np.where((pred2_class == 1) & (truth_class == 1))[0]) / len(np.where(truth_class == 1)[0])
f1 = 2*precision*recall / (precision + recall)
acc = 1 - (np.sum((pred2_class - truth_class)**2) / len(pred2_class))
    
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", acc)

Precision:  0.8331530194028318
Recall:  0.9606071228586127
F1:  0.8923519995940125
Accuracy:  0.895896724905


### Results on training with simple classification added:
Slight overall improvement on simple thresholding.<br><br>
Precision:  0.8331530194028318<br>
Recall:  0.9606071228586127<br>
F1:  0.8923519995940125<br>
Accuracy:  0.895896724905<br>

#### Try to add a couple of layers at the end and freeze the rest?

In [28]:
from keras.layers import Dense

model3 = model
model3.pop()
for i in range(len(model3.layers)-1):
    model3.layers[i].trainable = False
    
model3.add(Dense(units=256,activation='relu', name='new_layer_1'))    
model3.add(Dense(units=1,activation='sigmoid', name='new_output'))

In [29]:
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model3.fit(X, y_n, batch_size=64,verbose=1,epochs=1, validation_split=0.1)

Train on 440121 samples, validate on 48903 samples
Epoch 1/1
440121/440121 [==============================] - 797s 2ms/step - loss: 0.2631 - acc: 0.8986 - val_loss: 0.3055 - val_acc: 0.8503


In [32]:
pred3 = model3.predict(X)

In [42]:
model3.save('rectangle_classifier.h5')

In [36]:
pred3_class = np.zeros(len(pred3))
pred3_class[np.where(pred3 >= 0.5)[0]] += 1
truth_class = np.zeros(len(pred3))
truth_class[np.where(np.sum(signatures,axis=1) > 0)] += 1

precision = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where((pred3_class == 1))[0])
recall = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where(truth_class == 1)[0])
f1 = 2*precision*recall / (precision + recall)
acc = 1 - (np.sum((pred3_class - truth_class)**2) / len(pred3_class))

print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", acc)

Precision:  0.8407457939096458
Recall:  0.948210635576052
F1:  0.8912504412755805
Accuracy:  0.896060316058


### Results on training only last two layers (newly added), removing previous output layer:
Slightly better than the previous method (but much faster).<br><br>
Precision:  0.8407457939096458<br>
Recall:  0.948210635576052<br>
F1:  0.8912504412755805<br>
Accuracy:  0.896060316058<br>

In [9]:
pred_4 = model.predict(X)

In [10]:
print("Done")

Done


In [11]:
model.pop()
pred_5 = model.predict(X)

In [12]:
permutation = np.random.permutation(len(y_n))
X_p = X[permutation]
pred_4p = pred_4[permutation]
pred_5p = pred_5[permutation]
y_np = y_n[permutation]

### Using SVM on reclangle-net output

In [13]:
from sklearn.svm import SVC

kernels = ['linear', 'rbf', 'sigmoid']
scores =[]

for kernel in kernels:
    svm = SVC(kernel=kernel)
    svm = svm.fit(pred_4p[:50000],y_np[:50000])
    scores.append(svm.score(pred_4p[-50000:],y_np[-50000:]))
    
scores

[0.89216666666666666, 0.88983333333333337, 0.88706666666666667]

In [19]:
from sklearn.svm import LinearSVC

svm_lin = LinearSVC()
svm_lin.fit(pred_5p[:300000],y_np[:300000])
print(svm_lin.score(pred_5p[300000:],y_np[300000:]))

0.894045200609


### Using decision forest on trimmed rectangle net output

In [21]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree = tree.fit(pred_5p[:300000],y_np[:300000])
pred_tree = tree.predict(pred_5p[300000:])
print(tree.score(pred_5p[300000:], y_np[300000:]))

0.8468765871


In [20]:
tree = DecisionTreeClassifier()
tree = tree.fit(pred_4p[:300000],y_np[:300000])
pred_tree = tree.predict(pred_4[300000:])
print(tree.score(pred_4p[300000:], y_np[300000:]))

0.842728965634


### Using decision forest on raw data

In [41]:
X_p = X_p.reshape((X_p.shape[0],sample_length))

tree = DecisionTreeClassifier()
tree = tree.fit(X_p[:300000],y_np[:300000])
# pred_tree = tree.predict(pred_4[300000:])
print(tree.score(X_p[300000:], y_np[300000:]))

0.83989863721


### Using a smaller neural network

In [27]:
from keras.layers import Conv1D, Flatten
from keras.models import Sequential

layer1 = Conv1D(filters=16, input_shape = (sample_length,1,),kernel_size=4,
                activation='linear',padding='valid', strides=1)
layer2_b = Flatten()
layer5 = Dense(units=512,activation='relu')
layer6 = Dense(units=256,activation='relu')
outputs = Dense(units=1,activation='sigmoid', name='output_new')

model = Sequential()

model.add(layer1)
model.add(layer2_b)
model.add(layer5)
model.add(layer6)
model.add(outputs)

In [28]:
y_n[:,np.newaxis,np.newaxis].shape

(489024, 1, 1)

In [29]:
for l in model.layers:
    print(l.name)

conv1d_1
flatten_1
dense_4
dense_5
output_new


In [30]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model.fit(X, y_n, batch_size=64,verbose=1,epochs=5, validation_split=0.1)

Train on 440121 samples, validate on 48903 samples
Epoch 1/5
440121/440121 [==============================] - 95s 216us/step - loss: 0.2714 - acc: 0.8950 - val_loss: 0.3036 - val_acc: 0.8565
Epoch 2/5
440121/440121 [==============================] - 90s 205us/step - loss: 0.2472 - acc: 0.9055 - val_loss: 0.3321 - val_acc: 0.8414
Epoch 3/5
440121/440121 [==============================] - 90s 205us/step - loss: 0.2390 - acc: 0.9094 - val_loss: 0.3493 - val_acc: 0.8365
Epoch 4/5
440121/440121 [==============================] - 91s 207us/step - loss: 0.2324 - acc: 0.9122 - val_loss: 0.2982 - val_acc: 0.8591
Epoch 5/5
440121/440121 [==============================] - 95s 215us/step - loss: 0.2267 - acc: 0.9147 - val_loss: 0.2833 - val_acc: 0.8633


In [31]:
model.save('classifier.h5')

In [83]:
model = load_model('classifier.h5')

In [32]:
pred = model.predict(X)

In [33]:
pred_class = np.zeros(len(pred))
pred_class[np.where(pred >= 0.5)[0]] += 1

precision = len(np.where((pred_class == 1) & (truth_class == 1))[0]) / len(np.where((pred_class == 1))[0])
recall = len(np.where((pred_class == 1) & (truth_class == 1))[0]) / len(np.where(truth_class == 1)[0])
f1 = 2*precision*recall / (precision + recall)
acc = 1 - (np.sum((pred_class - truth_class)**2) / len(pred_class))

print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", acc)

Precision:  0.8610604256495077
Recall:  0.9645951224397816
F1:  0.9098919975092865
Accuracy:  0.91418417092


### Training on a small converted rectangle net

In [11]:
from keras.models import load_model
model2 = load_model('rectangles_small.h5')

In [13]:
from keras.layers import Dense

model2.pop()

for i in range(len(model2.layers) - 1):
    model2.layers[i].trainable = False

model2.add(Dense(units=1, activation='sigmoid'))

In [15]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model2.fit(X, y_n, batch_size=64,verbose=1,epochs=5, validation_split=0.1)

Train on 440121 samples, validate on 48903 samples
Epoch 1/5
440121/440121 [==============================] - 29s 66us/step - loss: 0.3488 - acc: 0.8819 - val_loss: 0.3651 - val_acc: 0.8654
Epoch 2/5
440121/440121 [==============================] - 25s 57us/step - loss: 0.3063 - acc: 0.8908 - val_loss: 0.3505 - val_acc: 0.8684
Epoch 3/5
440121/440121 [==============================] - 29s 67us/step - loss: 0.2988 - acc: 0.8921 - val_loss: 0.3345 - val_acc: 0.8695
Epoch 4/5
440121/440121 [==============================] - 28s 65us/step - loss: 0.2956 - acc: 0.8926 - val_loss: 0.3317 - val_acc: 0.8701
Epoch 5/5
440121/440121 [==============================] - 25s 57us/step - loss: 0.2939 - acc: 0.8930 - val_loss: 0.3604 - val_acc: 0.8612


In [17]:
pred3 = model2.predict(X)

In [18]:
pred3_class = np.zeros(len(pred3))
pred3_class[np.where(pred3 >= 0.5)[0]] += 1
truth_class = np.zeros(len(pred3))
truth_class[np.where(np.sum(signatures,axis=1) > 0)] += 1

precision = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where((pred3_class == 1))[0])
recall = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where(truth_class == 1)[0])
f1 = 2*precision*recall / (precision + recall)
acc = 1 - (np.sum((pred3_class - truth_class)**2) / len(pred3_class))

print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", acc)

Precision:  0.8476506246022975
Recall:  0.9217924146062761
F1:  0.8831682045488746
Accuracy:  0.890453229289


In [19]:
from keras.models import load_model
model3 = load_model('rectangles_small.h5')

In [21]:
from keras.layers import Dense

model3.pop()

model3.add(Dense(units=1, activation='sigmoid'))

In [22]:
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model3.fit(X, y_n, batch_size=64,verbose=1,epochs=5, validation_split=0.1)

Train on 440121 samples, validate on 48903 samples
Epoch 1/5
440121/440121 [==============================] - 87s 198us/step - loss: 0.2654 - acc: 0.8975 - val_loss: 0.2878 - val_acc: 0.8646
Epoch 2/5
440121/440121 [==============================] - 90s 206us/step - loss: 0.2471 - acc: 0.9057 - val_loss: 0.3005 - val_acc: 0.8527
Epoch 3/5
440121/440121 [==============================] - 90s 205us/step - loss: 0.2393 - acc: 0.9093 - val_loss: 0.3105 - val_acc: 0.8569
Epoch 4/5
440121/440121 [==============================] - 90s 204us/step - loss: 0.2336 - acc: 0.9115 - val_loss: 0.3389 - val_acc: 0.8447
Epoch 5/5
440121/440121 [==============================] - 88s 200us/step - loss: 0.2282 - acc: 0.9137 - val_loss: 0.3147 - val_acc: 0.8569


In [25]:
pred3 = model3.predict(X)

In [26]:
pred3_class = np.zeros(len(pred3))
pred3_class[np.where(pred3 >= 0.5)[0]] += 1
truth_class = np.zeros(len(pred3))
truth_class[np.where(np.sum(signatures,axis=1) > 0)] += 1

precision = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where((pred3_class == 1))[0])
recall = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where(truth_class == 1)[0])
f1 = 2*precision*recall / (precision + recall)
acc = 1 - (np.sum((pred3_class - truth_class)**2) / len(pred3_class))

print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", acc)

Precision:  0.8554001294696169
Recall:  0.9685193868678269
F1:  0.908451937160255
Accuracy:  0.912319231776


In [ ]:
model3.save('rectangle_small_classifier.h5')

In [35]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model2.fit(X, y_n, batch_size=64,verbose=1,epochs=10, validation_split=0.1)

Train on 440121 samples, validate on 48903 samples
Epoch 1/10
440121/440121 [==============================] - 31s 70us/step - loss: 0.3478 - acc: 0.8827 - val_loss: 0.3647 - val_acc: 0.8664
Epoch 2/10
440121/440121 [==============================] - 29s 67us/step - loss: 0.3063 - acc: 0.8906 - val_loss: 0.3305 - val_acc: 0.8735
Epoch 3/10
440121/440121 [==============================] - 27s 61us/step - loss: 0.2988 - acc: 0.8920 - val_loss: 0.3543 - val_acc: 0.8629
Epoch 4/10
440121/440121 [==============================] - 26s 60us/step - loss: 0.2956 - acc: 0.8927 - val_loss: 0.3587 - val_acc: 0.8614
Epoch 5/10
440121/440121 [==============================] - 25s 57us/step - loss: 0.2939 - acc: 0.8929 - val_loss: 0.3387 - val_acc: 0.8679
Epoch 6/10
440121/440121 [==============================] - 25s 57us/step - loss: 0.2928 - acc: 0.8932 - val_loss: 0.3721 - val_acc: 0.8578
Epoch 7/10
440121/440121 [==============================] - 28s 63us/step - loss: 0.2921 - acc: 0.8932 - val_

In [36]:
pred3 = model2.predict(X)

In [37]:
pred3_class = np.zeros(len(pred3))
pred3_class[np.where(pred3 >= 0.5)[0]] += 1
truth_class = np.zeros(len(pred3))
truth_class[np.where(np.sum(signatures,axis=1) > 0)] += 1

precision = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where((pred3_class == 1))[0])
recall = len(np.where((pred3_class == 1) & (truth_class == 1))[0]) / len(np.where(truth_class == 1)[0])
f1 = 2*precision*recall / (precision + recall)
acc = 1 - (np.sum((pred3_class - truth_class)**2) / len(pred3_class))

print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("Accuracy: ", acc)

Precision:  0.8445217088604972
Recall:  0.9277516514233426
F1:  0.884182347505022
Accuracy:  0.890827444052


In [63]:
model_dAE = load_model('../denoising/denoising.h5')

In [64]:
for i in range(len(model_dAE.layers)):
    model_dAE.layers[i].trainable = False

In [65]:
from keras.layers import Flatten
model_dAE.add(Flatten())
model_dAE.add(Dense(128, activation='relu'))
model_dAE.add(Dense(64, activation='relu'))
model_dAE.add(Dense(1, activation='sigmoid'))

In [66]:
model_dAE.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 32, 8)             40        
_________________________________________________________________
dense_1 (Dense)              (None, 32, 232)           2088      
_________________________________________________________________
dense_2 (Dense)              (None, 32, 64)            14912     
_________________________________________________________________
dense_3 (Dense)              (None, 32, 232)           15080     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 1)             929       
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               4224      
__________

In [67]:
model_dAE.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

training_history = model_dAE.fit(X, y_n, batch_size=64,verbose=1,epochs=5, validation_split=0.1)

Train on 440121 samples, validate on 48903 samples
Epoch 1/5
440121/440121 [==============================] - 154s 350us/step - loss: 0.2965 - acc: 0.8832 - val_loss: 0.3371 - val_acc: 0.8338
Epoch 2/5
440121/440121 [==============================] - 153s 348us/step - loss: 0.2747 - acc: 0.8928 - val_loss: 0.3169 - val_acc: 0.8417
Epoch 3/5
440121/440121 [==============================] - 152s 346us/step - loss: 0.2696 - acc: 0.8947 - val_loss: 0.3371 - val_acc: 0.8366
Epoch 4/5
440121/440121 [==============================] - 151s 343us/step - loss: 0.2664 - acc: 0.8960 - val_loss: 0.3525 - val_acc: 0.8373
Epoch 5/5
440121/440121 [==============================] - 154s 350us/step - loss: 0.2638 - acc: 0.8974 - val_loss: 0.3117 - val_acc: 0.8517
